In [10]:
import torch
from  torch import nn
from torchvision import models
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os
from matplotlib import pyplot as pp
import numpy as np
import torch.nn.functional as F
import cv2

In [11]:
l= pd.read_csv('Train_Labels.csv')


In [12]:
train_image_list = ['./Resized_train/'+file for file in os.listdir('./Resized_train')]

In [13]:
X_train,X_test,y_train,y_test = train_test_split(train_image_list,l,test_size=0.2)

In [14]:
train_labels = {}
for i,j in y_train.values:
    train_labels[i]=j

test_labels = {}
for i,j in y_test.values:
    test_labels[i]=j
    
labels = {}
for i,j in l.values:
    labels[i]=j


In [15]:
test_image_list =  ['./Resized_test/'+file for file in os.listdir('./Resized_test')]

In [16]:
class imagedataset(Dataset):
    
    def __init__(self,filelist, labels):
        self.filelist = filelist
        self.labels= labels
    
    def __len__(self):
        return len(self.filelist)
    
    def __getitem__(self, index):
        img = (cv2.imread(self.filelist[index] , 0))
        img = torch.from_numpy(img).type(torch.FloatTensor)
        img = torch.unsqueeze(img, 0)
        target = self.labels[self.filelist[index].split('/')[-1].split('.')[0]]
        #print(self.filelist[index],self.filelist[index].split('/')[-1])
        return (img, target)
    
class imagedataset_test(Dataset):
    
    def __init__(self,filelist):
        self.filelist = filelist
    
    def __len__(self):
        return len(self.filelist)
    
    def __getitem__(self, index):
        img = (cv2.imread(self.filelist[index] , 0))
        img = torch.from_numpy(img).type(torch.FloatTensor)
        img = torch.unsqueeze(img, 0)
        return (img,self.filelist[index].split('/')[-1].split('.')[0])

In [17]:
train_images_load = imagedataset(X_train, labels)
val_images_load = imagedataset(X_test, labels)
test_images_load = imagedataset_test(test_image_list)

In [18]:
train_data_loader = DataLoader(dataset=train_images_load, batch_size=8)
val_data_loader = DataLoader(dataset=val_images_load, batch_size=8)
test_data_loader = DataLoader(dataset=test_images_load, batch_size=8)

In [19]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)



In [20]:
class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(88, 16, kernel_size=5)
        self.conv4 = nn.Conv2d(88, 8, kernel_size=5)
        self.incept1 = InceptionA(in_channels=16)
        self.incept2 = InceptionA(in_channels=32)
        self.incept3 = InceptionA(in_channels=16)
        self.incept4 = InceptionA(in_channels=8)
        #self.conv5 = nn.Conv2d(8, 4, kernel_size=3)
        #self.conv6 = nn.Conv2d(4, 2, kernel_size=3)
        #self.conv7 = nn.Conv2d(2, 1, kernel_size=3)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8800,4000)
        self.fc2 = nn.Linear(4000,2000)
        self.fc3 = nn.Linear(2000,1000)
        self.fc4 = nn.Linear(1000,500)
        self.fc5 = nn.Linear(500,100)
        self.fc6 = nn.Linear(100,2)
        self.conv2_drop = nn.Dropout2d()
        
    def forward(self, x):
        insize = x.size(0)
        c1 = F.relu(self.maxpool(self.conv1(x)))
        #print(c1.shape)
        c1 = self.incept1(c1)
        #print(c1.shape)
        c2 = F.relu(self.maxpool(self.conv2_drop(self.conv2(c1))))
        c2 = self.incept2(c2)
        c3 = F.relu(self.maxpool(self.conv2_drop(self.conv3(c2))))
        c3 = self.incept3(c3)
        c4 = F.relu(self.maxpool(self.conv2_drop(self.conv4(c3))))
        c4 = self.incept4(c4)
        #c5 = self.maxpool(nn.functional.relu(self.conv5(c4)))
        #c6 = self.maxpool(nn.functional.relu(self.conv6(c5)))
        #c7 = self.maxpool(nn.functional.relu(self.conv7(c6)))
        c5 = c4.view(insize, -1)
        #print(c5.shape)
        l1 = self.fc1(c5)
        l1 = F.dropout(l1, training=self.training)
        l2 = self.fc2(l1)
        l2 = F.dropout(l2, training=self.training)
        l3 = self.fc3(l2)
        l3 = F.dropout(l3, training=self.training)
        l4 = self.fc4(l3)
        l4 = F.dropout(l4, training=self.training)
        l5 = self.fc5(l4)
        l5 = F.dropout(l5, training=self.training)
        l6 = self.fc6(l5)
        l6 = F.dropout(l6, training=self.training)
        f = F.softmax(l6)
        return f

In [21]:
model =  mynet()

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_data_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print("output: ",output.data.max(1, keepdim=True)[1],"target: ",target)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_data_loader.dataset),
                100. * batch_idx / len(train_data_loader), loss.data[0]))


def val():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in val_data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(val_data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_data_loader.dataset),
        100. * correct / len(val_data_loader.dataset)))


for epoch in range(1, 20):
    train(epoch)
    val()

/home/anand/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


output:  tensor([[ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 1]]) target:  tensor([ 0,  0,  0,  1,  1,  1,  1,  1])
Train Epoch: 1 [0/512 (0%)]	Loss: 0.687025


/home/anand/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


output:  tensor([[ 0],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 1]]) target:  tensor([ 1,  0,  0,  0,  0,  0,  0,  0])
Train Epoch: 1 [80/512 (16%)]	Loss: 1.110747
output:  tensor([[ 0],
        [ 1],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]) target:  tensor([ 1,  1,  1,  1,  0,  1,  1,  1])
Train Epoch: 1 [160/512 (31%)]	Loss: 0.580719
output:  tensor([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 0]]) target:  tensor([ 0,  1,  0,  1,  1,  0,  1,  1])
Train Epoch: 1 [240/512 (47%)]	Loss: 0.705719
output:  tensor([[ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 0]]) target:  tensor([ 1,  1,  1,  0,  0,  1,  0,  0])
Train Epoch: 1 [320/512 (62%)]	Loss: 1.110747
output:  tensor([[ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 0]]) target:

/home/anand/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/anand/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.1046, Accuracy: 61/128 (47%)

output:  tensor([[ 1],
        [ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]) target:  tensor([ 0,  0,  0,  1,  1,  1,  1,  1])
Train Epoch: 2 [0/512 (0%)]	Loss: 0.753204
output:  tensor([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 1]]) target:  tensor([ 1,  0,  0,  0,  0,  0,  0,  0])
Train Epoch: 2 [80/512 (16%)]	Loss: 0.783233
output:  tensor([[ 1],
        [ 0],
        [ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]) target:  tensor([ 1,  1,  1,  1,  0,  1,  1,  1])
Train Epoch: 2 [160/512 (31%)]	Loss: 0.860747
output:  tensor([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]) target:  tensor([ 0,  1,  0,  1,  1,  0,  1,  1])
Train Epoch: 2 [240/512 (47%)]	Loss: 0.908233
output:  tensor([[ 0],
        [ 0],
        [ 1],
        [ 1],
        [ 

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    pred={}
    n = []
    o = []
    for data,file in test_data_loader:
        data = Variable(data, volatile=True)
        output = model(data)
        # sum up batch loss
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        n.append(file)
        o.append([x[0] for x in pred])
    name = [item for sublist in n for item in sublist]
    out = [int(item) for sublist in o for item in sublist]
    print ([int(c) for c in out])
    df = pd.DataFrame({'Images':name,'Labels':out})
    df.to_csv('Results.csv',index=None)



In [ ]:
test()